In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import the necessary library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn import metrics
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import pickle
from sklearn.model_selection import validation_curve
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from itertools import chain

In [ ]:
df_review = pd.read_csv('/kaggle/input/amazon-music-reviews/Musical_instruments_reviews.csv')
df_review.head()

* You have to categorise opinions expressed in feedback forums

In [ ]:
# review the head of dataset
df_review.head()

In [ ]:
# checking the scale of overall rating
print('Maximum rating scale of overall rating {}'.format(df_review.overall.max()))
print('Minimum rating scale of overall rating {}'.format(df_review.overall.min()))

* **NOTE:We should the categorize the opnion of the feedback based on the overall rating because overall rating indicate that how much customers are satisfied from that product. <br>**

* Rating and there General Meaning are follows <br>
<pre>
    Rating            General Meaning
    1.0               I hate it.
    2.0               I don't like it.
    3.0               It's okey.
    4.0               I like it.
    5.0               I love it.
</pre>



In [ ]:
# defing the function for adding the category for the feedback
def add_category(df):

  if df['overall']==1.0:
    return "I hate it"
  elif df['overall']==2.0:
    return "I don't like it"
  elif df['overall'] == 3.0:
    return "It's okey"
  elif df['overall'] == 4.0:
    return "I like it"
  elif df['overall'] == 5.0:
    return "I love it"
  else:
    return -1


In [ ]:
# adding the category in dataframe
df_review['category'] = df_review.apply(add_category, axis=1)
plt.figure(figsize=(6,6))
plt.title('Percentage of category')
ax = sns.countplot(y = 'category', data = df_review)
total = len(df_review)
for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * p.get_width()/total)
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height()/2
        ax.annotate(percentage, (x, y))


+ **We can see the above graph**<br>
<pre>
  67.6% customers love the product.
  7.5% customers it's okey.
  20.3% customers like the product.
  2.4% customres don't like the product.
  2.1% customers hate the product.
</pre>

+ You have to classify individual comments/reviews and you have to determine overall rating based on individual comments/reviews.

In [ ]:
df_review.head()

In [ ]:
# checking the total number of nan values of each features
df_review.isnull().sum()

+ We can see the above only to two features has nan values reviewerName and reviewText respectively.
+ we can reviewText row whose values is nan because this is an important features. Curruently imput with missing string.

In [ ]:
# imputing the reviewText features with missing 
df_review['reviewText'] = df_review.reviewText.fillna('missing')

# concating the reviewText and summary features
df_review['review']= df_review['reviewText'] +" "+ df_review['summary']

# deleting the summary column
df_review.drop(['summary','reviewText'], inplace=True, axis=1)


In [ ]:
df_review.head()

* Our goal is predict the overall rating based on the comment/ review
* so review/comment is input feature and overall rating is the label
* This problem is mutliclass classification problem

In [ ]:
# create a dataframe that has only reviewText and overall rating
df = df_review[['review', 'overall']].copy()
df.head()

#### Text Preprocessing

In [ ]:
# defing the funtion for remoing the punctuation
def clean(text):

  # converting the all the text into lowercase
  text = str(text).lower()
  text = re.sub('\[.*?\]', '', text)
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub('\n', '', text)
  text = re.sub('\w*\d\w*', '', text)
  return text

In [ ]:
# apply the clean function on dataframe
df['review'] = df.review.apply(lambda x: clean(x))
df.head()

In [ ]:
nltk.download('stopwords')

In [ ]:
# remove the stop_words from the reivew
df['review']= df.review.apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords.words('english')]))
df.head()

In [ ]:
nltk.download('wordnet')

In [ ]:
# conveting the words into the root words using the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
df['review'] = df.review.apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))
df.head()

In [ ]:
# converting the words into base words using stemming method
ps = PorterStemmer()
df['review']= df.review.apply(lambda x: ' '.join([ps.stem(word) for word in x.split()]))
df.head()

In [ ]:
# featuers extraction most 5000 best features from the all featues with 1 to 3 gram using TF-IDF
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,3))
x = tfidf.fit_transform(df['review'])

In [ ]:
# label of featuere
y = df['overall']
print('Origonal no samples of each ratting {}'.format(Counter(y)))
print('original shape of the input features {}'.format(x.shape))
print('origonal shape of the output features {}'.format(y.shape))

+ we can easly see that five classess has different number of samples so we need to handle this
+ we are using the over_sampling technique to handle the imbalance dataset

In [ ]:
# create the SMOTE over_sampling technique object and fit_resample the features
sm = SMOTE()
x_res, y_res = sm.fit_resample(x, y)
print('Before sampling the  shape of dataset {}'.format(Counter(y)))
print('After sampling the shape of dataset {}'.format(Counter(y_res)))

In [ ]:
# create the test and train set
x_train, x_test, y_train, y_test = train_test_split(x_res, y_res, random_state=0, test_size=0.25)

In [ ]:
# model selection
# cross validation score
def cross_validation(model,x,y):

  cross_value_scored = []
  for model in models:
      model_name = model.__class__.__name__
      accuracies= cross_val_score(model, x, y, scoring = 'accuracy', cv = 5)
      for accuracy in accuracies:
          cross_value_scored.append((model_name, accuracy))
  df_cv = pd.DataFrame(cross_value_scored, columns =['model_name', 'accuracy'])
  acc = pd.concat([df_cv.groupby('model_name').accuracy.mean(),df_cv.groupby('model_name').accuracy.std()], axis= 1,ignore_index=True)
  acc.columns = ['Mean Accuracy', 'Standard deviation']
  return acc

In [ ]:
# define all the models
models = [
    RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0),
    XGBClassifier(),
    LinearSVC(),
    BernoulliNB(),
    KNeighborsClassifier()
    
]
# calling the cross_validation function
cross_validation(models, x_res, y_res)

In [ ]:
# Training the model and predict 
def train_predict_final_model(x_train, y_train, x_test):
  svc = LinearSVC(C=100.0, random_state=0)
  svc.fit(x_train, y_train)
  return svc, svc.predict(x_test)

In [ ]:
# call the train_predict_final_model
model, y_pred = train_predict_final_model(x_train, y_train, x_test)
print('Accuracy of the model {}'.format(accuracy_score(y_test, y_pred)))

+ Validate your build model, use any of your choice of validation matrices

In [ ]:
# plot the confusion matrix
def confusion_matrix_plot(y_test, y_pred):
  cm = confusion_matrix(y_test, y_pred)
  fig, ax = plt.subplots(figsize=(8,8))
  sns.heatmap(cm, annot=True, cmap = 'Reds',square = True,xticklabels = [1,2,3,4,5],yticklabels=[1,2,3,4,5])
  plt.ylabel('Actual')
  plt.xlabel('Predicted')
  plt.title('Confusion Matrix.')

# calling the confusion_matrix_plot function
confusion_matrix_plot(y_test, y_pred)

In [ ]:
print('Accuracy of the model {}'.format(accuracy_score(y_test, y_pred)))

In [ ]:
# report of the model
print('Report of the model {}'.format(classification_report(y_test, y_pred)))

In [ ]:
# ploting the learning curver
def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
   
       
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,train_sizes=train_sizes,return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,train_scores_mean + train_scores_std, alpha=0.1,color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,test_scores_mean + test_scores_std, alpha=0.1,color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt

In [ ]:
title = 'Learning curve of LinearSVC'
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)
estimator = LinearSVC(C=100.0)
plot_learning_curve(estimator, title, x_res, y_res,  ylim=(0.7, 1.01),cv=cv, n_jobs=4)

Perform an Exploratory Data Analysis for the Text Data (Reviews) and help the organisation to understand better about their customer feedbacks.

In [ ]:
# rating of the insturiments
plt.figure(figsize=(6,6))
plt.title('Rating of the Musical Instruments')
ax = sns.countplot(y = 'overall', data = df_review)
total = len(df_review)
for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * p.get_width()/total)
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height()/2
        ax.annotate(percentage, (x, y))

In [ ]:
all_words = df.review.values
negtive_words= df[df.overall <= 2.0]['review'].values
positive_words = df[df.overall >= 4.0]['review'].values

In [ ]:
# conert string of list to word of list
def string_to_word(words):
  word = []
  for w in words:
    word.append(w.split())
  return word

In [ ]:
# 10 most frequnt words
def most_frequent(word, title):
  word = string_to_word(word)
  word = list(chain.from_iterable(word))
  word = Counter(word)
  freq_words = word.most_common(10)

  y = [x[0] for x in freq_words]
  x = [x[1] for x in freq_words]
  plt.title(title)
  plt.bar(y, x)
  plt.show()

In [ ]:
most_frequent(all_words, '10 Most frequent words')

In [ ]:
most_frequent(negtive_words, '10 Most Negative words')

In [ ]:
most_frequent(positive_words, '10 most positive words')